In [5]:
!pip install azure-functions azure-core azure-ai-documentintelligence azure-storage-blob azure-search-documents


In [ ]:
import requests
import pandas as pd
import re
from azure.storage.blob import BlobServiceClient
import os
from dotenv import load_dotenv

load_dotenv()
# Azure Blob Storage 연결 문자열 및 컨테이너 이름
connect_str = os.getenv('CONNECT_STR')
container_name = 'config'

# BlobServiceClient 생성
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_client = blob_service_client.get_container_client(container_name)

# CSV 파일 경로 및 데이터 읽기
csv_file_path = 'C:/Users/sykim/Desktop/url/url_config.csv'
df = pd.read_csv(csv_file_path, encoding='euc-kr')

for index, row in df.iterrows():
    url = row['URL']
    product_nm = row['PRODUCT_NM']
    gbm = row['GBM']
    category_grp = row['CATEGORY_GRP']
    cdmodel = row['CDMODEL']

    # URL의 파일명 부분만 추출
    match = re.search(r'/([^/]+)\.pdf$', url)
    if match:
        url_filename = match.group(1)
    else:
        print(f"URL에서 파일명을 추출할 수 없습니다: {url}")
        continue  # URL이 유효하지 않으면 다음 행으로 넘어감

    # 파일명 지정
    url_filename = re.sub(r'[<>:"/\\|?*]', "_", url_filename)
    file_name = f"{url_filename}.pdf"
    file_name = file_name.replace("/", "_")  # 파일명에 '/'가 있으면 제거

    # PDF 다운로드
    response = requests.get(url)
    if response.status_code == 200:
        # Blob Client 생성
        blob_client = container_client.get_blob_client(file_name)

        try:
            # Blob에 PDF 파일 업로드 (대용량 파일 최적화 설정 추가)
            blob_client.upload_blob(
                response.content, 
                overwrite=True,
                max_block_size=2 * 1024 * 1024,    # 8 MiB
                max_single_put_size=4 * 1024 * 1024,
                max_concurrency=4  # 병렬 연결 수를 4로 설정하여 더 빠른 업로드 가능
            )
            print(f"파일이 업로드되었습니다: {file_name}")
        except Exception as e:
            print(f"파일 업로드 중 오류가 발생했습니다: {file_name}. 오류: {e}")
    else:
        print(f"URL에서 파일을 다운로드하지 못했습니다: {url}")


파일 업로드 중 오류가 발생했습니다: OID51746_IB_DV8700TK_SimpleUX_DV20B9760_KO_240105.pdf. 오류: ('Connection aborted.', TimeoutError('The write operation timed out'))
파일이 업로드되었습니다: DB68-11654A-00_QG_22Y_ACR_COMMON_KR_KO_220103-D01.pdf


KeyboardInterrupt: 

In [14]:
import requests
import pandas as pd
import re
import os
import shutil

# CSV 파일 경로 및 저장 경로
csv_file_path = 'C:/Users/sykim/Desktop/url_blob/merged_scms.csv'

# CSV 파일 읽기
df = pd.read_csv(csv_file_path, encoding='euc-kr')

In [15]:
df['url_check'] = df.duplicated(subset='URL', keep=False).map({True: 'O', False: 'X'})

In [16]:
duplicate_rows = df[df['url_check'] == 'O']

In [17]:
duplicate_rows

,SEQ,구분,GBM,CATEGORY_GRP,PRODUCT_NM,CDMODEL,URL,STDKEY,WRITELANG,url_check


In [ ]:
import pandas as pd
import re

# CSV 파일 경로 및 데이터 읽기
csv_file_path = 'C:/Users/sphong/hong/merged_scms.csv'
df = pd.read_csv(csv_file_path, encoding='euc-kr')

# URL에서 파일명 추출하여 새로운 열 'filename'에 추가
df['filename'] = df['URL'].apply(lambda x: re.search(r'/([^/]+\.pdf)$', x).group(1) if re.search(r'/([^/]+\.pdf)$', x) else None)

# 같은 파일명을 가진 행들의 CDMODEL 값을 결합하여 새로운 파일명 생성
df['new_filename'] = df.groupby('filename')['CDMODEL'].transform(lambda x: ','.join(x.unique()) + '_' + x.name if x.name else None)

# 결과 확인용으로 'SEQ', 'CDMODEL', 'filename', 'new_filename' 열을 출력
print(df[['SEQ', 'CDMODEL', 'filename', 'new_filename']])

# 필요 시, 변경된 파일명을 포함한 데이터를 새로운 CSV 파일로 저장
df.to_csv('C:/Users/sykim/Desktop/url/updated_url_config.csv', encoding='euc-kr', index=False)


In [21]:
csv_file_path = 'C:/Users/sykim/Desktop/url_blob/updated_url_config.csv'
df = pd.read_csv(csv_file_path, encoding='euc-kr')

In [23]:
df1 = df[['SEQ', 'CDMODEL', 'filename', 'new_filename']]

In [24]:
df1

,SEQ,CDMODEL,filename,new_filename
0,3378,SL-M4530ND,M4530_QIG_KOR.pdf,SL-M4530ND_M4530_QIG_KOR.pdf
1,3377,SL-M4030ND,M4030_QIG_KOR.pdf,SL-M4030ND_M4030_QIG_KOR.pdf
2,3366,SL-C3010ND,C301x_QIG3_korean_for_customer.pdf,SL-C3010ND_C301x_QIG3_korean_for_customer.pdf
3,3361,SL-C1454FW,CLX-419x_QIG1_worldwide_for_customer.pdf,SL-C1454FW_CLX-419x_QIG1_worldwide_for_custome...
4,3369,SL-M2079F,M207x_QIG3_korean_for_customer.pdf,SL-M2079F_M207x_QIG3_korean_for_customer.pdf
...,...,...,...,...
1654,"791, 792, 792","WA10CG5441BW, WA13CG5441BW, WA13CG5441BW",OID71048_IB_WA5000C_GOOD_KO_240614.pdf,"WA10CG5441BW, WA13CG5441BW, WA13CG5441BW_OID71..."
1655,"798, 798, 799","WA16CG6741BD, WA16CG6741BD, WA16CG6741BV",OID71050_IB_WA6000C_BETTER_KO_240611.pdf,"WA16CG6741BD, WA16CG6741BD, WA16CG6741BV_OID71..."
1656,399,RB33A3662AP,RB3000_DA68-04069G-02_SVC.pdf,RB33A3662AP_RB3000_DA68-04069G-02_SVC.pdf
1657,"1498, 1499, 1505, 1506, 1507, 1508, 1511, 1513...","NZ63DB607CABS, NZ63DB607CAWS, NZ63DB657CABS, N...",IB_FULL_NZ9000D_NZ63DB857_657_607_KO_240617.pdf,"NZ63DB607CABS, NZ63DB607CAWS, NZ63DB657CABS, N..."


In [35]:
excel_file_path = 'C:/Users/sykim/Desktop/url_blob/merged_scms_v0.3.xlsx'

# 엑셀 파일을 읽어 데이터프레임으로 변환
df_v2 = pd.read_excel(excel_file_path, sheet_name=0)

In [28]:
df_v2['filename'] = df_v2['URL'].apply(lambda x: re.search(r'/([^/]+\.pdf)$', x).group(1) if re.search(r'/([^/]+\.pdf)$', x) else None)

# 같은 파일명을 가진 행들의 CDMODEL 값을 결합하여 새로운 파일명 생성
df_v2['new_filename'] = df_v2.groupby('filename')['CDMODEL'].transform(lambda x: ','.join(x.unique()) + '_' + x.name if x.name else None)

In [30]:
df_v22 = df_v2[['CDMODEL', 'filename', 'new_filename']]

In [36]:
df_v2

,SEQ,구분,GBM,CATEGORY_GRP,PRODUCT_NM,CDMODEL,URL,STDKEY,WRITELANG
0,1,제품,PC/주변기기,프린터,흑백 레이저 프린터 45 ppm,SL-M4530ND,https://downloadcenter.samsung.com/content/EM/...,Quick Guide,KO
1,2,제품,PC/주변기기,프린터,흑백 레이저 프린터 40 ppm,SL-M4030ND,https://downloadcenter.samsung.com/content/EM/...,Quick Guide,KO
2,3,제품,PC/주변기기,프린터,컬러 레이저 프린터 30/30 ppm,SL-C3010ND,https://downloadcenter.samsung.com/content/EM/...,Quick Guide,KO
3,4,제품,PC/주변기기,프린터,컬러 레이저 복합기 14/14 ppm,SL-C1454FW,https://downloadcenter.samsung.com/content/EM/...,Quick Guide,EN
4,5,제품,PC/주변기기,프린터,흑백 레이저 복합기 20 ppm,SL-M2079F,https://downloadcenter.samsung.com/content/EM/...,Quick Guide,KO
...,...,...,...,...,...,...,...,...,...
1653,1654,제품,DA,세탁기,"그랑데 통버블 세탁기 14 kg, 그랑데 통버블 세탁기 16 kg, 그랑데 통버블 ...","WA14CG6441BY, WA16CG6441BW, WA16CG6441BY",https://downloadcenter.samsung.com/content/UM/...,User Manual,KO
1654,1655,제품,DA,세탁기,"그랑데 통버블 세탁기 10 kg, 그랑데 통버블 세탁기 13 kg, 그랑데 통버블 ...","WA10CG5441BW, WA13CG5441BW, WA13CG5441BW",https://downloadcenter.samsung.com/content/UM/...,User Manual,KO
1655,1656,제품,DA,세탁기,그랑데 통버블 세탁기 16 kg,"WA16CG6741BD, WA16CG6741BD, WA16CG6741BV",https://downloadcenter.samsung.com/content/UM/...,User Manual,KO
1656,1657,제품,DA,냉장고,BESPOKE 냉장고 2도어 키친핏 333 L,RB33A3662AP,https://downloadcenter.samsung.com/content/UM/...,User Manual (Common),KO


In [37]:
import requests
import pandas as pd
import re
import os
import shutil
from tqdm import tqdm

# CSV 파일 경로 및 저장 경로
excel_file_path = 'C:/Users/sykim/Desktop/url_blob/merged_scms_v0.3.xlsx'
save_dir = 'C:/Users/sykim/Desktop/url_blob/pdf_file'

# 디렉토리가 존재하지 않으면 생성
os.makedirs(save_dir, exist_ok=True)

# CSV 파일 읽기
df = pd.read_excel(excel_file_path, sheet_name=0)

# URL 컬럼을 순회하면서 PDF 다운로드 및 파일명 저장
for index, row in tqdm(df.iterrows()):
    url = row['URL']
    seq = row['SEQ']  # SEQ 값을 가져옴

    # 정규식을 사용해 URL의 파일명 부분만 추출
    match = re.search(r'/([^/]+)\.pdf$', url)
    if match:
        url_filename = match.group(1)  # url_filename을 파일명으로 사용
    else:
        print(f"URL에서 파일명을 추출할 수 없습니다: {url}")
        continue

    # 파일명에서 허용되지 않는 문자 제거 (Windows 시스템 호환)
    url_filename = re.sub(r'[<>:"/\\|?*]', "_", url_filename)

    # SEQ 값을 포함한 파일명 지정
    file_name = f"{seq}_{url_filename}.pdf"
    file_path = os.path.join(save_dir, file_name)

    try:
        # PDF 다운로드 및 저장
        response = requests.get(url, stream=True)  # 스트림 모드로 다운로드하여 효율적 저장
        if response.status_code == 200:
            with open(file_path, 'wb') as file:
                shutil.copyfileobj(response.raw, file)  # 빠른 저장을 위해 shutil 사용
            print(f"파일이 저장되었습니다: {file_path}")
        else:
            print(f"URL에서 파일을 다운로드하지 못했습니다: {url}")
    except Exception as e:
        print(f"파일 다운로드 또는 저장 중 오류 발생: {url}. 오류: {e}")
    finally:
        response.close()  # 리소스 해제


1it [00:19, 19.11s/it]

파일이 저장되었습니다: C:/Users/sykim/Desktop/url_blob/pdf_file\1_M4530_QIG_KOR.pdf


2it [00:38, 19.45s/it]

파일이 저장되었습니다: C:/Users/sykim/Desktop/url_blob/pdf_file\2_M4030_QIG_KOR.pdf


3it [00:53, 17.16s/it]

...

파일이 저장되었습니다: C:/Users/sykim/Desktop/url_blob/pdf_file\1656_OID71050_IB_WA6000C_BETTER_KO_240611.pdf


1657it [1:53:42,  3.05s/it]

파일이 저장되었습니다: C:/Users/sykim/Desktop/url_blob/pdf_file\1657_RB3000_DA68-04069G-02_SVC.pdf


1658it [1:53:48,  4.12s/it]

파일이 저장되었습니다: C:/Users/sykim/Desktop/url_blob/pdf_file\1658_IB_FULL_NZ9000D_NZ63DB857_657_607_KO_240617.pdf
